In [16]:
from bs4 import BeautifulSoup
import os
import pandas as pd

In [17]:
current_path = os.getcwd()
fb_folder = "../SEC_8K/Data/FB"

In [18]:
def find_text(file):
    HtmlFile = open(file, 'r', encoding='utf-8')
    soup = BeautifulSoup(HtmlFile, 'html.parser')
#     html = ""
#     tag = soup.find("hr").next
#     while True:
#         if tag.name == "hr":
#             break
#         else:
#             try:
#                 html += tag.text
#             except:
#                 pass
#             tag = tag.next

    [s.extract() for s in soup(['style', 'script', '[document]', 'head', 'title'])]
    visible_text = soup.getText()
            
    return visible_text

In [19]:
os.chdir(fb_folder)

all_text = {}

for date in os.listdir():
    os.chdir(date)
    text = find_text("8K.html")
    all_text[date] = [text]
    os.chdir("..")

os.chdir(current_path)

In [20]:
df = pd.DataFrame.from_dict(all_text).T.squeeze()

In [21]:
df = df.str.replace("\n", "")
df = df.str.replace("\xa0", "")
df = df.str.replace("\x93", "")
df = df.str.replace("\x94", "")
df = df.str.replace("\x92", "")
df = df.str.replace("/s/", "")

In [28]:
df.head()

2012-06-25    8-K1d371533d8k.htmFORM 8-K UNITED STATES SECUR...
2012-07-26    8-K1d385430d8k.htmFORM 8-K UNITED STATES SECUR...
2012-09-04    8-K1d405186d8k.htmFORM 8-K UNITED STATES SECUR...
2012-10-15    8-K1d424570d8k.htmFORM 8-K UNITED STATES SECUR...
2012-10-23    8-K1d427739d8k.htmFORM 8-K UNITED STATES SECUR...
Name: 0, dtype: object

In [29]:
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")
from gensim.models import Doc2Vec
from sklearn import utils
import gensim
from gensim.models.doc2vec import TaggedDocument
import re
from gensim.models import doc2vec

In [40]:
def process(fname):
    for i, line in enumerate(fname):
        yield gensim.models.doc2vec.TaggedDocument(gensim.utils.simple_preprocess(line), [i])


In [45]:
corpus = list(process(df))

In [56]:
corpus[0]

TaggedDocument(words=['htmform', 'united', 'states', 'securities', 'and', 'exchange', 'commission', 'washington', 'form', 'current', 'report', 'pursuant', 'to', 'section', 'or', 'of', 'the', 'securities', 'exchange', 'act', 'of', 'date', 'of', 'report', 'date', 'of', 'earliest', 'event', 'reported', 'june', 'facebook', 'inc', 'exact', 'name', 'of', 'registrant', 'as', 'specified', 'in', 'charter', 'delaware', 'state', 'or', 'other', 'jurisdiction', 'commission', 'irs', 'employerof', 'incorporation', 'file', 'number', 'identification', 'no', 'willow', 'roadmenlo', 'park', 'california', 'address', 'of', 'principal', 'executive', 'offices', 'zip', 'code', 'registrants', 'telephone', 'number', 'including', 'area', 'code', 'former', 'name', 'or', 'former', 'address', 'if', 'changed', 'since', 'last', 'report', 'check', 'the', 'appropriate', 'box', 'below', 'if', 'the', 'form', 'filing', 'is', 'intended', 'to', 'simultaneously', 'satisfy', 'the', 'filing', 'obligation', 'of', 'the', 'registr

In [57]:
model = gensim.models.doc2vec.Doc2Vec(vector_size=50, min_count=2, epochs=40)

In [58]:
model.build_vocab(corpus)

In [59]:
model.train(corpus, total_examples=model.corpus_count, epochs=model.epochs)

In [78]:
for i in range(len(df)):
    df[i] = model.infer_vector(corpus[i].words)

In [80]:
df

2012-06-25    [0.40349245, -1.1994313, 0.13297047, 0.3166482...
2012-07-26    [0.3179282, -0.6705323, -0.41433606, 0.6988617...
2012-09-04    [2.056731, -0.3734913, 1.7694564, 0.39918527, ...
2012-10-15    [2.0227444, -2.1397638, 0.373905, 0.35142675, ...
2012-10-23    [0.8838345, -0.71241564, -0.22185303, 0.418000...
2012-10-29    [0.78238785, -1.3628035, -0.08994775, 0.151832...
2012-11-29    [0.5910992, -1.8778521, -0.14869998, 0.2326103...
2013-01-30    [0.7029326, -0.8309122, -0.4415216, 0.3366325,...
2013-03-06    [0.92133987, -1.206244, -0.39933792, 1.2488497...
2013-04-03    [0.80154896, -1.1662692, -0.29937288, 0.231282...
2013-04-26    [0.5351618, -1.1124167, -0.043819346, 0.444213...
2013-05-01    [0.772674, -1.1505969, -0.45659646, 0.59379756...
2013-06-13    [-0.4627143, -0.9198941, -0.1874736, 0.5362691...
2013-07-24    [0.60996443, -0.71331036, -0.41182578, 0.56936...
2013-08-15    [1.4259973, -1.5411619, 0.22774364, 0.07839040...
2013-10-16    [0.38144523, -0.54302096, 

In [82]:
df.to_hdf("../SEC_8K/Data/8K data.h5", key='df', mode='w')

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\generic.py:1471: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->values] [items->None]

  return pytables.to_hdf(path_or_buf, key, self, **kwargs)
